 # Visualizing India’s nightlight change from 1992 to 2025 using DMSP and VIIRS # satellite datasets.

In [1]:
# ===============================================
# Nightlight Time-lapse for India (1992–2025)
# Combines DMSP-OLS (1992–2013) and VIIRS (2012–2025)
# Outputs MP4 + annotated GIF showing light growth
# ===============================================

# --- Install required Python packages ---
!pip -q install earthengine-api imageio imageio-ffmpeg pillow

# --- Import libraries ---
import ee, requests, imageio.v3 as iio
from PIL import Image as PILImage, ImageDraw, ImageFont
from IPython.display import Image as IPyImage
import os
from google.colab import userdata

# --- Import Google Cloud Project ID , linked to Google Earth Engine Account  ---
# Create your own via thsi link :
# I have stored the GCP ID as Colab Secret and used below. You can directly pass your GCP credentials instead

PROJECT_ID = userdata.get('GCP_ID')
ee.Authenticate()
ee.Initialize(project=PROJECT_ID)

# ===============================================
# SECTION 1: Load India boundary
# ===============================================

# Load GAUL Level 0 (countries)
admin0 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0")

# Select India
IND = admin0.filter(ee.Filter.eq('ADM0_NAME', 'India')).geometry()

# Print area for info
print("India area (sq km):", IND.area().divide(1e6).getInfo())

# --- STUDENT: Change 'India' to any country name (e.g., 'Indonesia', 'Nepal') to try other regions ---


# ===============================================
# SECTION 2: Load Nightlight Datasets
# ===============================================

start_year, end_year = 1992, 2025

# --- DMSP-OLS (stable lights, annual composites, DN 0–63) ---
dmsp_all = (ee.ImageCollection("NOAA/DMSP-OLS/NIGHTTIME_LIGHTS")
            .select("stable_lights")
            .filterBounds(IND)
            .filter(ee.Filter.calendarRange(1992, 2013, 'year')))

# --- VIIRS (monthly avg radiance, float, 2012+) ---
viirs_all = (ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG")
             .select("avg_rad")
             .filterBounds(IND)
             .filter(ee.Filter.calendarRange(2012, end_year, 'year')))

# --- NOTE: VIIRS starts in 2012, DMSP ends in 2013 → we blend for 2012–13


# ===============================================
# SECTION 3: Robust normalization (0–1 scaling)
# ===============================================

# Compute 2nd–98th percentile for each sensor across India
def robust_scale_params(ic: ee.ImageCollection, band: str, scale_m=500):
    p = ic.reduce(ee.Reducer.percentile([2, 98]))  # image with p2, p98 bands
    stats = p.reduceRegion(ee.Reducer.first(), IND, scale=scale_m, maxPixels=1e10, tileScale=4)
    p2  = ee.Number(stats.get(f"{band}_p2"))
    p98 = ee.Number(stats.get(f"{band}_p98"))

    # Handle nulls (if no data): fallback to default values
    p2  = ee.Algorithms.If(p2, p2, 0)
    p98 = ee.Algorithms.If(p98, p98, 1)
    return ee.Number(p2), ee.Number(p98)

# Normalize both sensors to 0–1
dmsp_p2, dmsp_p98 = robust_scale_params(dmsp_all, "stable_lights")
viirs_p2, viirs_p98 = robust_scale_params(viirs_all, "avg_rad")

# Normalize image using precomputed p2–p98 (clamp outside range)
def norm01(img, p2, p98):
    return img.subtract(p2).divide(ee.Number(p98).subtract(p2)).clamp(0, 1)

# Smooth the 0–1 image (optional: to make GIF prettier)
def smooth01(img01):
    return img01.focal_mean(radius=200, units='meters').clamp(0,1)

# Blend two images (e.g., DMSP + VIIRS in 2012–13)
def blend(a, b, w=0.5):
    return ee.Image(a).multiply(1.0 - w).add(ee.Image(b).multiply(w))


# ===============================================
# SECTION 4: Create annual RGB images (EE side)
# ===============================================

def year_nl_rgb(y):
    y = ee.Number(y)
    start = ee.Date.fromYMD(y, 1, 1)
    end   = start.advance(1, 'year')

    # Filter each sensor's image collection by year
    dmsp_ic  = dmsp_all.filterDate(start, end)
    viirs_ic = viirs_all.filterDate(start, end)

    # Check if we have images for this year
    dmsp_has  = dmsp_ic.size().gt(0)
    viirs_has = viirs_ic.size().gt(0)

    # Compute median image for that year (if any)
    dmsp_img  = dmsp_ic.median()
    viirs_img = viirs_ic.median()

    # Normalize (if present), else return masked 0
    dmsp01 = ee.Image(ee.Algorithms.If(
        dmsp_has, norm01(dmsp_img.select('stable_lights'), dmsp_p2, dmsp_p98),
        ee.Image(0).updateMask(ee.Image(0))
    ))
    viirs01 = ee.Image(ee.Algorithms.If(
        viirs_has, norm01(viirs_img.select('avg_rad'), viirs_p2, viirs_p98),
        ee.Image(0).updateMask(ee.Image(0))
    ))

    # Select which image to use
    before_2012 = y.lt(2012)
    blend_years = y.gte(2012).And(y.lte(2013))  # blend 2012 & 2013 for smoothness

    # If both sensors exist, blend them
    blended = ee.Image(ee.Algorithms.If(
        dmsp01.mask().And(viirs01.mask()),
        blend(dmsp01, viirs01, 0.5),
        ee.Image(ee.Algorithms.If(viirs_has, viirs01, dmsp01))
    ))

    # Final 0–1 image for this year
    nl01 = ee.Image(ee.Algorithms.If(
        before_2012, dmsp01,
        ee.Algorithms.If(blend_years, blended, viirs01)
    ))

    # Apply smoothing + fallback to black if null
    nl01 = ee.Image(ee.Algorithms.If(nl01, smooth01(nl01), ee.Image.constant(0)))

    # Convert to RGB for visualization
    vis = nl01.visualize(min=0, max=1, palette=[
        '#000000', '#ff6f00', '#ffff66', '#ffffff'  # black → orange → yellow → white
    ])
    return vis.rename(['R','G','B']).unmask(0).uint8().set('system:time_start', start.millis())


# Build image collection of yearly visual RGB frames
years = ee.List.sequence(start_year, end_year)
nl_ic = ee.ImageCollection(years.map(year_nl_rgb))


# ===============================================
# SECTION 5: Export MP4 video from Earth Engine
# ===============================================

# Set video dimensions and export region
video_params = {
    'dimensions': 720,
    'region': IND.bounds().getInfo()['coordinates'],  # full bounding box
    'framesPerSecond': 3  # 3 fps → faster playback
}

# Get video URL and download it
video_url = nl_ic.getVideoThumbURL(video_params)
#print("Video URL:", video_url)

# Save locally
mp4_path = "india_nightlights_1992_2025.mp4"
r = requests.get(video_url)
with open(mp4_path, "wb") as f:
    f.write(r.content)
print("MP4 saved:", mp4_path)

# ===============================================
# SECTION 6: Convert MP4 to GIF with year labels
# ===============================================

# Load frames from MP4
frames = iio.imread(mp4_path)

# Add year label on each frame
labeled = []
for i, frame in enumerate(frames):
    img = PILImage.fromarray(frame)
    draw = ImageDraw.Draw(img)
    try:
        font = ImageFont.truetype("DejaVuSans-Bold.ttf", 44)
    except:
        font = ImageFont.load_default()
    year = str(start_year + i)
    draw.text((20, img.height - 70), year, font=font,
              fill="white", stroke_width=3, stroke_fill="black")
    labeled.append(img)

# Save as animated GIF
gif_path = "india_nightlights_1992_2025.gif"
labeled[0].save(gif_path, save_all=True, append_images=labeled[1:],
                duration=int(1000 / video_params['framesPerSecond']), loop=0)

print("GIF saved:", gif_path)
IPyImage(filename=gif_path)

Output hidden; open in https://colab.research.google.com to view.